In [2]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd

In [3]:
spark=SparkSession.builder.master("local[*]").appName('test2').getOrCreate()

In [4]:
df2=spark.read.parquet("fhv/2021/01/")

In [5]:
df2.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_flag: string (nullable = true)



In [6]:
df2.head(1)

[Row(hvfhs_license_num='438598', dispatching_base_num='HV0005', pickup_datetime=None, dropoff_datetime=datetime.datetime(2021, 1, 2, 5, 56, 37), PULocationID=None, DOLocationID=75, SR_flag='41')]

In [8]:
df2.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID')\
.filter(df2.hvfhs_license_num=='438598').show()

+---------------+-------------------+------------+------------+
|pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+---------------+-------------------+------------+------------+
|           NULL|2021-01-02 05:56:37|        NULL|          75|
+---------------+-------------------+------------+------------+



In [17]:
from pyspark.sql import functions as F
from pyspark.sql import types

In [ ]:
F.to_date()

exploring UDFs

In [14]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [15]:
crazy_stuff('B02884')

's/b44'

In [18]:
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [28]:
df2\
    .withColumn('pickup_date', F.to_date(df2.pickup_datetime))\
    .withColumn('dropoff_date', F.to_date(df2.dropoff_datetime))\
    .withColumn('base_id', crazy_stuff_udf(df2.dispatching_base_num))\
    .select('pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID')\
    .show()

+-----------+------------+------------+------------+
|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-----------+------------+------------+------------+
|       NULL|  2021-01-02|        NULL|          75|
|       NULL|  2021-01-05|        NULL|          67|
|       NULL|  2021-01-04|        NULL|          91|
|       NULL|  2021-01-02|        NULL|          87|
|       NULL|  2021-01-01|        NULL|          17|
|       NULL|  2021-01-03|        NULL|         167|
|       NULL|  2021-01-03|        NULL|          92|
|       NULL|  2021-01-02|        NULL|         255|
|       NULL|  2021-01-03|        NULL|          17|
|       NULL|  2021-01-04|        NULL|         169|
|       NULL|  2021-01-02|        NULL|          33|
|       NULL|  2021-01-02|        NULL|          47|
|       NULL|  2021-01-01|        NULL|          61|
|       NULL|  2021-01-01|        NULL|         242|
|       NULL|  2021-01-02|        NULL|         207|
|       NULL|  2021-01-04|        NULL|       